# Decision Trees

In [9]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from public_tests import *

<a name="2"></a>
## 2 -  Problem Statement

Suppose you are starting a company that grows and sells wild mushrooms. 
- Since not all mushrooms are edible, you'd like to be able to tell whether a given mushroom is edible or poisonous based on it's physical attributes
- You have some existing data that you can use for this task. 

Can you use the data to help you identify which mushrooms can be sold safely? 

Note: The dataset used is for illustrative purposes only. It is not meant to be a guide on identifying edible mushrooms.



<a name="3"></a>
## 3 - Dataset

You will start by loading the dataset for this task. The dataset you have collected is as follows:

| Cap Color | Stalk Shape | Solitary | Edible |
|:---------:|:-----------:|:--------:|:------:|
|   Brown   |   Tapering  |    Yes   |    1   |
|   Brown   |  Enlarging  |    Yes   |    1   |
|   Brown   |  Enlarging  |    No    |    0   |
|   Brown   |  Enlarging  |    No    |    0   |
|   Brown   |   Tapering  |    Yes   |    1   |
|    Red    |   Tapering  |    Yes   |    0   |
|    Red    |  Enlarging  |    No    |    0   |
|   Brown   |  Enlarging  |    Yes   |    1   |
|    Red    |   Tapering  |    No    |    1   |
|   Brown   |  Enlarging  |    No    |    0   |


-  You have 10 examples of mushrooms. For each example, you have
    - Three features
        - Cap Color (`Brown` or `Red`),
        - Stalk Shape (`Tapering` or `Enlarging`), and
        - Solitary (`Yes` or `No`)
    - Label
        - Edible (`1` indicating yes or `0` indicating poisonous)

<a name="3.1"></a>
### 3.1 One hot encoded dataset
For ease of implementation, we have one-hot encoded the features (turned them into 0 or 1 valued features)

| Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|:---------:|:--------------------:|:--------:|:------:|
|     1     |           1          |     1    |    1   |
|     1     |           0          |     1    |    1   |
|     1     |           0          |     0    |    0   |
|     1     |           0          |     0    |    0   |
|     1     |           1          |     1    |    1   |
|     0     |           1          |     1    |    0   |
|     0     |           0          |     0    |    0   |
|     1     |           0          |     1    |    1   |
|     0     |           1          |     0    |    1   |
|     1     |           0          |     0    |    0   |

Therefore,
- `X_train` contains three features for each example 
    - Brown Color (A value of `1` indicates "Brown" cap color and `0` indicates "Red" cap color)
    - Tapering Shape (A value of `1` indicates "Tapering Stalk Shape" and `0` indicates "Enlarging" stalk shape)
    - Solitary  (A value of `1` indicates "Yes" and `0` indicates "No")

- `y_train` is whether the mushroom is edible 
    - `y = 1` indicates edible
    - `y = 0` indicates poisonous

In [2]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

In [3]:
X_train.shape

(10, 3)

In [4]:
y_train.shape

(10,)

In [5]:
X_train[:4, :]

array([[1, 1, 1],
       [1, 0, 1],
       [1, 0, 0],
       [1, 0, 0]])

In [6]:
y_train[:4]

array([1, 1, 0, 0])

### The steps for making the decision treee are  
- Start by all examples at the root node
- Calculate information gain for splitting on all possible features, and pick the one with the highest information gain
- Split dataset according to the selected feature, and create left and right branches of the tree
- Keep repeating splitting process until stopping criteria is met

### First we will implement the `compute_entropy()` function  
This function will take the target values as input and will give the calculated entropy as the output.  
The entropy can be calculated by the formula  
$$ H(p_1) = -p_1\text{log}_2(p_1) - (1 - p_1)\text{log}_2(1 - p_1)$$

In [10]:
def compute_entropy(y):
    if len(y) != 0:
        p1 = len(y[y==1]) / len(y)
        if p1 != 0 and p1 != 1:
            entropy = -p1 * np.log2(p1) - (1 - p1) * np.log2(1 - p1)
            return entropy
    return 0

In [11]:
print("Entropy at root node: ", compute_entropy(y_train)) 

# UNIT TESTS
compute_entropy_test(compute_entropy)

Entropy at root node:  1.0
 All tests passed.


### Create a function `split_dataset` to split the dataset into right and left subparts  
- The function will take the features, list of indices and feature as input
- The function will return the indices corresponding the left and right indices

In [12]:
def split_dataset(X, node_indices, feature):
    left_indices = []
    right_indices = []
    
    for index in node_indices:
        if X[index, feature] == 1:
            left_indices.append(index)
        else:
            right_indices.append(index)
    return left_indices, right_indices

In [13]:
root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

feature = 0

left_indices, right_indices = split_dataset(X_train, root_indices, feature)

print("Left indices: ", left_indices)
print("Right indices: ", right_indices)

# UNIT TESTS    
split_dataset_test(split_dataset)

Left indices:  [0, 1, 2, 3, 4, 7, 9]
Right indices:  [5, 6, 8]
 All tests passed.


### Create a fuction `compute_information_gain` which will calculate the information gain for every step  
- The information gain takes the features, target, node_indices and feature as the input
- The function returns the information gain for split.

The formula for calculating the information gain is  
$$\text{Information Gain} = H(p_1^{node}) - (w^{left}H(p_1^{left}) + w^{right}H(p_1^{right}))$$  
where  
$H(p_1^{node})$ = Entropy of the node which is to be splitted  
$w^{left}$ and $w^{right}$ are the proportions of the examples on left and right, respectively.  
$H(p_1^{left})$ and $H(p_1^{right})$ are the entropy of examples on left and right, respectively.

In [25]:
def compute_information_gain(X, y, node_indices, feature):
    # split the dataset
    left_indices, right_indices = split_dataset(X, node_indices, feature)

    # Get the features and targets for splitted dataset to pass them as argument to the funcitons
    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]

    # Calculate Entopies
    node_entropy = compute_entropy(y_node)
    l_node_entropy = compute_entropy(y_left) 
    r_node_entropy = compute_entropy(y_right)

    # Calculate proportions of the splitted data
    w_left = len(X_left) / len(X_node)
    w_right = len(X_right) / len(X_node)

    # Calculate the information gain
    information_gain = node_entropy - (w_left * l_node_entropy + w_right * r_node_entropy)

    return information_gain

In [26]:
info_gain0 = compute_information_gain(X_train, y_train, root_indices, feature=0)
print("Information Gain from splitting the root on brown cap: ", info_gain0)
    
info_gain1 = compute_information_gain(X_train, y_train, root_indices, feature=1)
print("Information Gain from splitting the root on tapering stalk shape: ", info_gain1)

info_gain2 = compute_information_gain(X_train, y_train, root_indices, feature=2)
print("Information Gain from splitting the root on solitary: ", info_gain2)

# UNIT TESTS
compute_information_gain_test(compute_information_gain)

Information Gain from splitting the root on brown cap:  0.034851554559677034
Information Gain from splitting the root on tapering stalk shape:  0.12451124978365313
Information Gain from splitting the root on solitary:  0.2780719051126377
 All tests passed.


### Write a function `get_best_split` which will get the best split by calculating information gain for all the possible splits on the features

In [19]:
def get_best_split(X, y, node_indices):
    features = X.shape[1]
    best_feature = -1
    max_info_gain = 0
    for i in range(features):
        info_gain = compute_information_gain(X, y, node_indices, i)
        if info_gain > max_info_gain:
            max_info_gain = info_gain
            best_feature = i
    return best_feature

In [20]:
best_feature = get_best_split(X_train, y_train, root_indices)
print("Best feature to split on: %d" % best_feature)

# UNIT TESTS
get_best_split_test(get_best_split)

Best feature to split on: 2
 All tests passed.


## Now as all the helper functions are ready lets build the `Decision Tree`

In [28]:
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return

    best_feature = get_best_split(X, y, node_indices)
    tree.append((current_depth, branch_name, node_indices, best_feature))

    formatting = "-"*current_depth
    print("%s Depth %d, %s : split on feature: %d" % (formatting, current_depth, branch_name, best_feature))
    
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    build_tree_recursive(X, y, left_indices, "Left", 2, current_depth + 1)
    build_tree_recursive(X, y, right_indices, "Rigth", 2, current_depth + 1)

In [29]:
build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)

 Depth 0, Root : split on feature: 2
- Depth 1, Left : split on feature: 0
  -- Left leaf node with indices [0, 1, 4, 7]
  -- Rigth leaf node with indices [5]
- Depth 1, Rigth : split on feature: 1
  -- Left leaf node with indices [8]
  -- Rigth leaf node with indices [2, 3, 6, 9]
